## **Ingesta del archivo `constructors.json`**

In [ ]:
# Creamos un parametro para el nombre del archivo
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../utils/configuration"

In [ ]:
# Llamamos al notebook que contiene funciones comunes
%run "../utils/common_functions"

### Paso 1 - Leer el archivo JSON

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import lit

In [ ]:
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
constructor_df = spark.read \
.schema(constructors_schema) \
.json(f"{raw_folder_path}/constructors.json")
#.json(f"{raw_folder_path}/{v_file_date}/constructors.json")

### Paso 2 - Eliminar las columnas no deseadas

In [ ]:
constructor_dropped_df = constructor_df.drop(col('url'))

### Paso 3 - Cambiar el nombre de las columnas y añadir "ingestion date"

In [ ]:
constructor_renamed_df = constructor_dropped_df.withColumnRenamed("constructorId", "constructor_id") \
                                               .withColumnRenamed("constructorRef", "constructor_ref") \
                                               .withColumn("data_source", lit(v_data_source)) \
                                               .withColumn("file_date", lit(v_file_date))

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
# La fecha de ingestión será del tipo timestamp
constructor_final_df = add_ingestion_date(constructor_renamed_df)

### Paso 4 - Escribir datos en el datalake como parquet y crear la tabla **constructors** en la base de datos **f1_processed**

In [ ]:
# Recordar que Databricks utiliza Hive para la gestión de tablas
# Esto quiere decir que una tabla en Hive se asocia con un directorio específico que contiene archivos de datos
# Aunque los datos subyacentes son solo archivos en un sistema de archivos, Hive proporciona una interfaz que presenta 
# estos datos como tablas.
# La base de datos tambien es un directorio. Podemos crearla manualmente, como con lenguaje SQL.
# Hive permite a los usuarios interactuar con esos datos utilizando un lenguaje similar a SQL
# En resumen, solo se esta creando un directorio y un archivo
# Escribimos el archivo con formato PARQUET en la base de datos "f1_processed" y en la tabla "constructors"
constructor_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.constructors")

In [ ]:
dbutils.notebook.exit("Success")